# Work in progress

In [47]:
!pip3 install numpy --pre torch --force-reinstall --index-url https://download.pytorch.org/whl/nightly/cu116

Looking in indexes: https://download.pytorch.org/whl/nightly/cu116, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://download.pytorch.org/whl/nightly/numpy-1.24.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
^C


In [2]:
!pip install datasets transformers einops sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 50.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 K

In [3]:
import numpy as np
import math
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch
from einops import rearrange
from datasets import load_dataset
from transformers import AutoTokenizer, BertTokenizer

tokenizer_pl = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")
tokenizer_en = BertTokenizer.from_pretrained('bert-base-cased')

In [11]:
class TextDataset(Dataset):
    def __init__(self, split, transform=None, target_transform=None) -> None:
        super().__init__()
        
        dataset = load_dataset("opus_euconst", 'en-pl')

        if split == 'train':
            self.text_en = [tokenizer_en(text['en'], padding='max_length', return_tensors='pt', truncation=True, max_length=50)['input_ids'] for text in dataset['train']['translation'][:9000]]
            self.text_pl = [tokenizer_pl(text['pl'], padding='max_length', return_tensors='pt', truncation=True, max_length=50)['input_ids'] for text in dataset['train']['translation'][:9000]]

            self.mask_en = [tokenizer_en(text['en'], padding='max_length', return_tensors='pt', truncation=True, max_length=50)['attention_mask'] for text in dataset['train']['translation'][:9000]]
            self.mask_pl = [tokenizer_pl(text['pl'], padding='max_length', return_tensors='pt', truncation=True, max_length=50)['attention_mask'] for text in dataset['train']['translation'][:9000]]
            
        elif split == 'validation':
            self.text_en = [tokenizer_en(text['en'], padding='max_length', return_tensors='pt', truncation=True, max_length=50)['input_ids'] for text in dataset['train']['translation'][9000:]]
            self.text_pl = [tokenizer_pl(text['pl'], padding='max_length', return_tensors='pt', truncation=True, max_length=50)['input_ids'] for text in dataset['train']['translation'][9000:]]

            self.mask_en = [tokenizer_en(text['en'], padding='max_length', return_tensors='pt', truncation=True, max_length=50)['attention_mask'] for text in dataset['train']['translation'][9000:]]
            self.mask_pl = [tokenizer_pl(text['pl'], padding='max_length', return_tensors='pt', truncation=True, max_length=50)['attention_mask'] for text in dataset['train']['translation'][9000:]]
        else:
            raise ValueError

        del dataset

    def __len__(self):
        return len(self.text_en)

    def __getitem__(self, idx):
        return self.text_en[idx], self.text_pl[idx], self.mask_en[idx], self.mask_pl[idx]

In [12]:
train_ds = TextDataset('train')
valid_ds = TextDataset('validation')

Generating train split:   0%|          | 0/9938 [00:00<?, ? examples/s]

Dataset opus_euconst downloaded and prepared to /root/.cache/huggingface/datasets/opus_euconst/en-pl/1.0.0/d1e611a011f28fdda67a97024820e0a3813b4e4decca194d9a20b3207a39b908. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
torch.__version__

'2.0.0.dev20230202+cu116'

In [14]:
class SelfAttention(nn.Module):
    # from https://github.com/karpathy/nanoGPT

    def __init__(self, causal, n_embd, n_head, dropout=0.1):
        super().__init__()
        assert n_embd % n_head == 0
        self.causal = causal
        self.n_head = n_head
        self.n_embd = n_embd
        self.dropout = dropout
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(n_embd, 3 * n_embd)
        # output projection
        self.c_proj = nn.Linear(n_embd, n_embd)
        # regularization
        self.attn_dropout = nn.Dropout(dropout)
        self.resid_dropout = nn.Dropout(dropout)


    def forward(self, x, attn_mask=None):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        q, k ,v  = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=attn_mask, dropout_p=self.dropout, is_causal=self.causal)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.c_proj(y))
        return y

class MLP(nn.Module):
    # from https://github.com/karpathy/nanoGPT

    def __init__(self, n_embd, dropout):
        super().__init__()
        self.c_fc    = nn.Linear(n_embd, 4 * n_embd)
        self.c_proj  = nn.Linear(4 * n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.c_fc(x)
        x = torch.relu(x)
        x = self.c_proj(x)
        x = self.dropout(x)
        return x

class Block(nn.Module):
    # from https://github.com/karpathy/nanoGPT

    def __init__(self, causal, n_embd, n_head, dropout):
        super().__init__()
        self.ln_1 = nn.LayerNorm(n_embd)
        self.attn = SelfAttention(causal, n_embd, n_head, dropout)
        self.ln_2 = nn.LayerNorm(n_embd)
        self.mlp = MLP(n_embd, dropout)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

In [30]:
# batch size = 1300 gives approximately 25k tokens per batchm but memory issues
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True, drop_last=True)
valid_loader = DataLoader(valid_ds, batch_size=128, shuffle=False, drop_last=True)

In [31]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [32]:
@torch.no_grad()
def estimate_loss(train_loader, valid_loader, loss_func):
    model.eval()

    train_loss = 0
    for x, target, mask1, mask2 in train_loader:
        x, target, mask1, mask2 = x.to(device), target.to(device), mask1.to(device), mask2.to(device)
        preds = model(x)
        train_loss += loss_func(preds, target.ravel())
    train_loss /= len(train_loader)

    valid_loss = 0
    for x, target, mask1, mask2 in valid_loader:
        x, target, mask1, mask2 = x.to(device), target.to(device), mask1.to(device), mask2.to(device)
        preds = model(x)
        valid_loss += loss_func(preds, target.ravel())
    valid_loss /= len(valid_loader)

    model.train()
    return train_loss, valid_loss

In [33]:
def get_learning_rate(d_model, step_num, warmup_steps=4000):
    return d_model ** (-0.5) * min(step_num ** (-0.5), step_num * warmup_steps ** (-1.5))

In [39]:
class Transformer(nn.Module):
    def __init__(self, vocab_size, n_embd, output_dim, block_size, n_layer, n_head, dropout=0.1) -> None:
        super().__init__()
        self.vocab_size = vocab_size
        self.n_embd = n_embd
        self.output_dim = output_dim
        self.emb = nn.Embedding(num_embeddings=vocab_size, embedding_dim=n_embd)
        self.encoder = nn.ModuleList([Block(False, n_embd, n_head, dropout) for _ in range(n_layer)])
        self.output = nn.Linear(n_embd, output_dim)
        pos_emb = self._create_pos_emb(block_size, n_embd)
        self.register_buffer('pos_emb', pos_emb)

    @staticmethod
    def _create_pos_emb(block_size, hidden_size):
        pos_enc = []
        for pos in range(block_size):
            for i in range(hidden_size):
                if i % 2 == 0:
                    pos_enc.append(torch.sin(torch.tensor(pos / (10000 ** (2 * i / hidden_size )))))
                else:
                    pos_enc.append(torch.cos(torch.tensor(pos / (10000 ** (2 * i / hidden_size )))))
        pos_enc = torch.tensor(pos_enc).reshape(block_size, hidden_size)
        return pos_enc
    
    def forward(self, x):
        # B T
        x = self.emb(x)
        x = x + self.pos_emb
        x = x.squeeze()
        for block in self.encoder:
            x = block(x)
        # B T C
        x = self.output(x)
        x = x.squeeze()
        x = rearrange(x, 'B T C -> (B T) C')
        x = torch.softmax(x, dim=1)
        return x

In [42]:
d_model = 512
model = Transformer(vocab_size=tokenizer_en.vocab_size, n_embd=d_model, output_dim=tokenizer_pl.vocab_size, block_size=50, n_layer=2, n_head=2, dropout=0.1)
model.to(device)

Transformer(
  (emb): Embedding(28996, 512)
  (encoder): ModuleList(
    (0-1): 2 x Block(
      (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (attn): SelfAttention(
        (c_attn): Linear(in_features=512, out_features=1536, bias=True)
        (c_proj): Linear(in_features=512, out_features=512, bias=True)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (mlp): MLP(
        (c_fc): Linear(in_features=512, out_features=2048, bias=True)
        (c_proj): Linear(in_features=2048, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (output): Linear(in_features=512, out_features=50000, bias=True)
)

In [43]:
%%time
loss_func = nn.CrossEntropyLoss(label_smoothing=0.1)

torch.manual_seed(42)
losses = []
for i in range(2):

    lr = get_learning_rate(d_model, i + 1)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=[.9, .98], eps=1e-09)

    for x, target, mask1, mask2 in train_loader:
        x, target, mask1, mask2 = x.to(device), target.to(device), mask1.to(device), mask2.to(device)
        target = target.squeeze().flatten()
        optimizer.zero_grad(set_to_none=True)
        preds = model(x)
        loss = loss_func(preds, target)
        loss.backward()
        optimizer.step()
    train_loss, valid_loss = estimate_loss(train_loader, valid_loader, loss_func)
    print(f"epoch {str(i + 1).zfill(2)}: train loss {train_loss:.6f}, val loss {valid_loss:.6f}")

epoch 01: train loss 10.819724, val loss 10.819720
epoch 02: train loss 10.819670, val loss 10.819648
CPU times: user 1min 25s, sys: 189 ms, total: 1min 25s
Wall time: 1min 26s
